# Pandas

With Pandas (= panel data), tabular data can be handled very conveniently. More at https://pandas.pydata.org/.

Very helpful:
- https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html
- https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

Pandas uses two important data structures:
- `Series` are 1-dimensional arrays. In contrast to Numpy arrays, however, they have an additional index that can be used to conveniently access entries. Internally, this is based on Numpy arrays.
- `DataFrame` is an entire table consisting of several series (=columns) of the same length.

In [1]:
import numpy as np
import pandas as pd # empfehlenswerte Konvention

#### DataFrames

To manually store data in table we need to create DataFrame
- dictionary keys will be used as headers
- values in list as columns 
of the DataFrame

In [2]:
df = pd.DataFrame(
    {
        "Name": ["Alex","Martin","Den"],
        "Age": [28,22,35],
        "Sex": ["Female","Male","Male"]
    }
)
df

,Name,Age,Sex
0,Alex,28,Female
1,Martin,22,Male
2,Den,35,Male


Every column is a `Series`.

In [3]:
df["Age"]

0    28
1    22
2    35
Name: Age, dtype: int64

The first column of a DataFrame (or series) is the so-called index. It contains the *row names*. By default, these are the numbers 0, 1, 2, ...
However, you can also set a different index.

In [14]:
df = df.set_index("Name")
df


,Age,Sex
Name,,
Alex,28,Female
Martin,22,Male
Den,35,Male


We can create Series from scratch as well. Note that Series do not have column labels but they do have row labels

In [16]:
ages = pd.Series([19,22,34], name="Age")
ages
# This is the way to add row labels
ages.index = ["person1","person2","person3"]
ages

person1    19
person2    22
person3    34
Name: Age, dtype: int64

#### Read and write data

Pandas offers a variety of options for reading data from databases (via SQL) or from files, e.g. csv, xlsx, parquet, arrow, feather, ...

In [19]:
star_wars = pd.read_csv("starwars.csv")
star_wars

,name,height,mass,sex,eye_color,homeworld,jedi
0,Luke Skywalker,172,77.0,male,blue,Tatooine,True
1,R2-D2,96,32.0,none,red,Naboo,False
2,Darth Vader,202,136.0,male,yellow,Tatooine,True
3,Leia Organa,150,49.0,female,brown,Alderaan,True
4,Chewbacca,228,112.0,male,blue,Kashyyyk,False
5,Yoda,66,17.0,male,brown,NaN,True
6,Boba Fett,183,78.2,male,brown,Kamino,False


Gain initial insight into the data.
- I wanna see first 3 rows of a DataFrame: use the head() method with the required number of rows (in this case 3) as argument.
- If we are interested technical summary of a DataFrame we call info method on our DataFrame

In [21]:
star_wars.head(3)
star_wars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       7 non-null      object 
 1   height     7 non-null      int64  
 2   mass       7 non-null      float64
 3   sex        7 non-null      object 
 4   eye_color  7 non-null      object 
 5   homeworld  6 non-null      object 
 6   jedi       7 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 524.0+ bytes


Reading and writing xlsx files is also no problem. Here is an example of saving the data set as xlsx. The *openpyxl* package must be installed for this.

#### Selection of columns or rows

If you select not just one column name but a list of several columns, you get a DataFrame.

When selecting rows, you usually want to select rows that fulfill certain properties.
To do this, write a true/false vector of the length of the DataFrame in the square brackets `[]`.
Comparison operators (`<`, `>`, `==`, `<=`, `>=`, `!=`) and Boolean operators `&` (and), `|` (or) and `~` (not) are often very helpful.

If you want to select certain rows and columns *in one step*, `.loc` and `.iloc` are suitable.

Using `.loc[rows, columns]` (= location) you can address entries via their **row index**.

#### Create, remove and rename columns

When calculating new columns, the power of vectorization becomes clear.
The calculations are performed element by element for each entry of the vector.
An explicit loop is not necessary.

If you want to perform complicated calculations, you can write a function and apply it line by line using `map()` or `apply()`. 
line by line. More on this later! 

To rename columns, pass a dictionary with entries of the form `{"old_colname": "new_colname"}` or a suitable string function to the `.rename` function.

#### Visualization

(Here only very briefly. Visualization gets an extra session).

In [120]:
import matplotlib.pyplot as plt

#### Sorting

Sorting the dataframe has consequences for the index. 
If you also want the index to be ordered, you can either reset the index or have it ignored directly during sorting. 

#### Aggregation of data

Typical aggregation functions such as mean(), sum(), max(), quantile() ... can be called directly as a method of `Series`.

Very often you want to perform such aggregations in groups, i.e. for each unique value of a feature. This is known as **Split-Apply-Combine**: The overall table is split into individual tables according to the unique values of a feature, the aggregation function is calculated for each such table and then these values are collected in a smaller table.

We have just used a square bracket with column names directly after `.groupby()`. This is often done and restricts the individual tables to the named columns. The aggregation function then only affects these columns.

You can also group by several characteristics. The resulting object then has a MultiIndex.

Very often you want to count how often a value occurs in a column. This provides the basis for bar charts.
(Note: By default, NA values do not form a separate group and are not listed. However, this is often relevant in the context of data preparation. For this purpose, you can set the option `dropna=False` in `.groupby()` or `.value_counts()`).
As the resulting object is not a DataFrame, a `.reset_index()` is suitable for making the index an explicit column and connecting sorting steps, for example.

There are both `.count()` and `.size()`. These are very similar. While `.size()` returns the number of rows, `.count()` returns the number of non-NA values. One function therefore counts NA values, while the other does not.

#### Merging (=joining) several tables

Very often you want to combine several tables to enrich information.
Pandas can also be used to combine several DataFrames, as in SQL.

The following table shows the development of student numbers for the TH Aschaffenburg.
Unfortunately, only the course abbreviations are included and there is also no assignment to faculties.
The second table provides this information.

Using `merge()` you can perform all kinds of JOIN operations in Pandas, as we know them from SQL.
Here we want to enrich the table *th_small* with the respective degree program information.

With `merge()` you can also perform other joins (RIGHT JOIN, INNER JOIN, OUTER JOIN), for example by specifying `how="outer"`.

The easiest way to combine two DataFrames is to simply append one to the other.
Of course, this only makes sense if the columns match (or are at least subsets of each other).
For example, let's look at the data for the degree programs *MEDS* and *SD*.
With `concat()`, the two tables can be appended.

#### Data wrangling for advanced users: Long format and wide format

<!--- Data source:
- Destatis, Table 12411-0015: Population: districts, reference date (selection: all reference dates). (Download flat file!)
- Destatis, Table 12411-0018: Population: districts, reference date, sex, age groups (selection: all reference dates but only the 3 districts of Aschaffenburg, Würzburg and Schweinfurt). (Download Flat-File!) -->

There are various ways of displaying the same data in tabular form.
Depending on the application, it is necessary to switch between these forms. This is sometimes known as “pivoting”.
The best way to see this is with an example. 

<!--- The following table (adapted from Destatis table 12411-0016) shows the population development of the three cities Aschaffenburg, Schweinfurt and Würzburg.-->

For the sake of simplicity, we will initially disregard the total number of students and limit ourselves to the degree programs *BW* (Business Administration), *SD* (Software Design) and *EIT* (Electrical Engineering and Information Technology) since the winter semester 2020.

The table above contains one row for each combination of academic year and subject. 
This is called **long format**.
For time series in particular, however, it is often clearer to arrange the data differently.
In this case, we want to display the enrolment figures for each subject in a separate column.
This is then the **wide format**.

With the function `pivot()` you can “rotate” the fields appropriately.
Here you must specify which columns are to be retained as an index, which column contains the new column names and which are the actual values.
The wide format may seem clearer, but has the disadvantage that you have to change the table schema when new study programs are added.

Of course, you can also convert from wide format to long format.
This works with `melt()`.
(First, however, we use `reset_index()` to make the index a regular column).
In the argument `id_vars` we list all columns that are to be kept constant - all other columns are "pivoted" to two new columns, in that each combination of column name and respective entry forms a new line. 

We had just restricted the data set to the number of freshmen and removed the *students* column.
So we only let `pivot()` pivot the column *freshmen*.

If, on the other hand, we leave the student numbers in the data set, we can also pivot them.
This results in a **MultiIndex** for the columns.

Closely related to `pivot()` and `melt()` are the `stack()` and `unstack()` functions.
They are particularly useful in combination with indexes and MultiIndexes.

- `stack()` pivots all column labels (except the index) and returns a series (or a DataFrame) with an additional inner index level.
- `unstack()` pivots the innermost index level to column labels and thus creates a DataFrame.

For the sake of clarity, let's restrict ourselves to the beginner numbers above.

`stack()` rotates all column names to a new inner index level and thus returns a series with a two-level multi-index.

Using `unstack()`, the innermost level of a MultiIndex becomes new column names and we get the original DataFrame again.
(If you want to pivot another index level instead of the innermost level, you can specify an index level using both `stack` and `unstack`).

By the way: Instead of calling `pivot` you can also use `set_index` to create a MultiIndex and then call `unstack`.